In [2]:
import pandas as pd
import json
import openai
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os

from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
# === Étape 1 : Préparer les descriptions ===
product_data = pd.read_json('meta.jsonl', lines=True)


# Extraire les descriptions de chaque produit
descriptions = product_data['description'].fillna("").tolist()
 
# === Étape 2 : Initialiser les embeddings ===
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

In [ ]:
# Configurer le séparateur

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,  # Augmentez la taille des chunks
    chunk_overlap=50  # Réduisez légèrement le chevauchement
)


In [28]:
# Diviser les descriptions en chunks
segmented_texts = []
for description in descriptions:
    if description:  # Si la description existe
        chunks = text_splitter.split_text(" ".join(description))
        segmented_texts.extend(chunks)
 
print(f"Nombre total de chunks créés : {len(segmented_texts)}")
print("Exemple de chunk :", segmented_texts[:2])

Nombre total de chunks créés : 906
Exemple de chunk : ["JUST LOOK, You can tell the difference. Make everyday more convenient, it is slim but has big rooms. If you are looking for a rich and luxurious appearance, look no further. These double shoulders are the perfect leather for creating attractive finished belts, straps and wallets. It doesn't only show the perfect weight for accessories where rugged durability is needed but also has a natural finish and coarse grain.", 'Case does not need to be removed for charging. Camera opening allows unobstructed use of camera and flash. DandyCase proudly presents the premium "PERFECT PATTERN" from the line of stylish cases that will make your friends jealous! Stand out from the rest and show off your Apple iPhone 6 Plus with these one of a kind cases. These cases are made of a durable, yet slightly flexible, TPU material that won\'t stretch or tear and the patterns wont chip, fade, or peel. DandyCase PERFECT PATTERN cases come with a']


In [30]:
from langchain.vectorstores import Chroma
CHROMA_DB_DIR = "./chroma_db"  # Répertoire pour stocker la base Chroma
 
vectorstore = Chroma.from_texts(
    texts=segmented_texts,
    embedding=hf_embeddings,
    persist_directory=CHROMA_DB_DIR  # Permet de sauvegarder la base
)

In [44]:
retriever = vectorstore.as_retriever()

In [ ]:
# Charger le modèle pour la génération
HF_LLM_MODEL = "bigscience/bloomz-560m"  # Exemple de modèle
tokenizer = AutoTokenizer.from_pretrained(HF_LLM_MODEL)
model = AutoModelForCausalLM.from_pretrained(HF_LLM_MODEL)

In [ ]:
# la question à poser
query = "Tell me about OnePlus 6T"

In [ ]:
# Récupérer les documents pertinents
retrieved_docs = retriever.get_relevant_documents(query)

# Afficher les passages pertinents des documents récupérés
print("Passages pertinents :")
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i + 1}:\n{doc.page_content}\n")


Passages pertinents :
Document 1:
a resilient glass back, the OnePlus 6T was crafted with care and purpose. Experience a 6.41 inch Optic AMOLED display for true immersion through an 86% screen-to-body ratio, beautifully slim cut-out, and more. Nail every detail night or day with Optical

Document 2:
a resilient glass back, the OnePlus 6T was crafted with care and purpose. Experience a 6.41 inch Optic AMOLED display for true immersion through an 86% screen-to-body ratio, beautifully slim cut-out, and more. Nail every detail night or day with Optical

Document 3:
a resilient glass back, the OnePlus 6T was crafted with care and purpose. Experience a 6.41 inch Optic AMOLED display for true immersion through an 86% screen-to-body ratio, beautifully slim cut-out, and more. Nail every detail night or day with Optical

Document 4:
gets you up and running in just half an hour. Our operating system is all about ensuring your phone works for you, not the other way around. Powered by Android Pie, 

In [48]:
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [ ]:
# Créer le prompt personnalisé
prompt = f"""
    Voici des informations extraites des documents suivants :
    {context}

    Répondez à la question ci-dessous uniquement à partir des informations contenues dans ces documents. Si la réponse n'est pas disponible, dites-le clairement.

    Question : {query}
    Réponse :
"""


In [53]:
# Préparer les entrées pour le modèle
inputs = tokenizer(prompt, return_tensors="pt", truncation=False, max_length = 512)
    
# Générer une réponse
outputs = model.generate(inputs["input_ids"], max_new_tokens = 500, max_length=512)

# Décoder la réponse
raw_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

Both `max_new_tokens` (=500) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [54]:
# Extraire uniquement la réponse après "reply:"
if "reply:" in raw_response:
    clean_response = raw_response.split("reply:")[1].strip()  # Garder tout ce qui suit "reply:"
else:
    clean_response = raw_response.strip()  # Si "reply:" absent, tout afficher après le prompt

# Afficher uniquement la réponse nettoyée
print(f"Réponse :\n{clean_response}")


Réponse :
Voici des informations extraites des documents suivants :
    a resilient glass back, the OnePlus 6T was crafted with care and purpose. Experience a 6.41 inch Optic AMOLED display for true immersion through an 86% screen-to-body ratio, beautifully slim cut-out, and more. Nail every detail night or day with Optical

a resilient glass back, the OnePlus 6T was crafted with care and purpose. Experience a 6.41 inch Optic AMOLED display for true immersion through an 86% screen-to-body ratio, beautifully slim cut-out, and more. Nail every detail night or day with Optical

a resilient glass back, the OnePlus 6T was crafted with care and purpose. Experience a 6.41 inch Optic AMOLED display for true immersion through an 86% screen-to-body ratio, beautifully slim cut-out, and more. Nail every detail night or day with Optical

gets you up and running in just half an hour. Our operating system is all about ensuring your phone works for you, not the other way around. Powered by Android Pie